# Claim Detection

- In this notebook, I prepared a batch to check if each comment contains a valid, check-worthy claim.  
- I loaded all thesis-relevant comments and built a batch JSONL file using the rewritten comment text.  
- The prompt asked GPT-4o to reply with just `1` for yes (claim present) or `0` for no.  
- Because of the token limit, I split the batch into two parts before submitting it to OpenAI’s Batch API.  
- Once the outputs were ready, I downloaded the JSONL files and extracted the `custom_id` and claim response.  
- I saved the extracted results to a CSV and counted how many comments were labelled as containing a claim (`1`) and how many were not (`0`).


### Importing Libraries

In [ ]:
import pandas as pd
import json
from tqdm import tqdm
import os
import openai

In [2]:
# Loading the full dataset
df = pd.read_csv("Thesis_Relevant_With_Transcript_Influence_and_Agreement.csv")
print(f"Loaded {len(df)} comments.")

Loaded 71441 comments.


/var/folders/nv/wkg5hpgj1kl1012mfw_qwby80000gn/T/ipykernel_14104/3311419059.py:4: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Thesis_Relevant_With_Transcript_Influence_and_Agreement.csv")


### Creating the Json File

In [10]:
# Setting the output file name for the JSONL batch
output_jsonl = "batch_step_claim_detection.jsonl"

# Opening the file and writing each row as a separate task
with open(output_jsonl, "w", encoding="utf-8") as f:
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Building Claim Detection batch (Binary 1 = yes, 0 = no)"):
        comment_id = str(row["Comment_ID"])
        comment = row["Rewritten Comment"]

        # Creating the request payload for GPT-4o
        task = {
            "custom_id": comment_id,  # Track the comment by its ID
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "gpt-4o",
                "messages": [
                    {
                        "role": "system",
                        "content": "You are helping a researcher determine whether a YouTube comment contains a check-worthy factual claim."
                    },
                    {
                        "role": "user",
                        "content": f"""Comment: "{comment}"\n\nDoes this comment contain a verifiable factual claim?\n\nReply ONLY with:\n1 = yes\n0 = no\n\nReply with just the number: 1 or 0"""
                    }
                ],
                "temperature": 0,  
                "max_tokens": 5    
            }
        }


        f.write(json.dumps(task) + "\n")

print(f"Saved: {output_jsonl}")

Building Claim Detection batch (Binary 1 = yes, 0 = no): 100%|█| 71441/71441 [00:00<00:00]
Saved: batch_step_claim_detection.jsonl


### Splitting Batch due to Token Limit

In [16]:
input_file = "batch_step_claim_detection.jsonl" 
output_file_1 = input_file.replace(".jsonl", "_part1.jsonl")
output_file_2 = input_file.replace(".jsonl", "_part2.jsonl")

# Reading the file
with open(input_file, "r", encoding="utf-8") as f:
    lines = f.readlines()

# Splitting
mid = len(lines) // 2
part1 = lines[:mid]
part2 = lines[mid:]

# Writing to two new files
with open(output_file_1, "w", encoding="utf-8") as f:
    f.writelines(part1)

with open(output_file_2, "w", encoding="utf-8") as f:
    f.writelines(part2)

print(f"Saved {len(part1)} → {output_file_1}")
print(f"Saved {len(part2)} → {output_file_2}")


Saved 35720 → batch_step_claim_detection_part1.jsonl
Saved 35721 → batch_step_claim_detection_part2.jsonl


### Submitting Batches to Openai

In [18]:
# Setting my Openai Key
openai.api_key = "************"
client = openai.OpenAI(api_key=openai.api_key)

# Your JSONL batch files (submitted one after the other with no wait)
batch_files = [
    "batch_step_claim_detection_part1.jsonl",
    "batch_step_claim_detection_part2.jsonl"
]

# Submitting batches
for i, file_path in enumerate(batch_files):
    try:
        print(f"\nSubmitting batch {i+1}/{len(batch_files)}: {file_path}")

        # Uploading file
        with open(file_path, "rb") as f:
            upload = client.files.create(file=f, purpose="batch")
        print(f"Uploaded: File ID = {upload.id}")

        # Creating batch job
        batch = client.batches.create(
            input_file_id=upload.id,
            endpoint="/v1/chat/completions",
            completion_window="24h"
        )
        print(f"Batch submitted: Batch ID = {batch.id}")

    except Exception as e:
        print(f"Failed to submit {file_path}: {e}")


Submitting batch 1/2: batch_step_claim_detection_part1.jsonl
Uploaded: File ID = file-RRvpCRsPw7NQ4ZPYj3XjUv
Batch submitted: Batch ID = batch_67e5ec8b173c8190bf268370938960a2

Submitting batch 2/2: batch_step_claim_detection_part2.jsonl
Uploaded: File ID = file-KoTFC4TusJ8admJgvKpZXg
Batch submitted: Batch ID = batch_67e5ec9407dc8190a7b1932560c885a3


### Downloading Json Output from Openai

In [18]:
openai.api_key = "*************"

# List of output file IDs from completed batches
output_file_ids = [
    "file-2LUjxme1c4bQxUwqQrseJU",
    "file-LB5qVVZ8eacaUiAJyAwgd8"
]

# Downloading each output file properly
for file_id in output_file_ids:
    file_response = openai.files.content(file_id)

    # Saving the file locally in binary mode
    output_filename = f"{file_id}.jsonl"
    with open(output_filename, "wb") as f:
        for chunk in file_response.iter_bytes():
            f.write(chunk)
    
    print(f"File downloaded: {output_filename}")

File downloaded: file-2LUjxme1c4bQxUwqQrseJU.jsonl
File downloaded: file-LB5qVVZ8eacaUiAJyAwgd8.jsonl


### Converting Json Output to a CSV file

In [20]:
jsonl_files = [
    "file-2LUjxme1c4bQxUwqQrseJU.jsonl",
    "file-LB5qVVZ8eacaUiAJyAwgd8.jsonl"
]

# Storing extracted data
extracted_data = []

# Looping through files
for jsonl_file in jsonl_files:
    if not os.path.exists(jsonl_file):
        print(f"{jsonl_file} not found")
        continue

    print(f"Processing {jsonl_file}")

    with open(jsonl_file, "r", encoding="utf-8") as f:
        for line in f:
            json_obj = json.loads(line)

            custom_id = json_obj.get("custom_id", "")
            response = json_obj.get("response", {}).get("body", {}).get("choices", [{}])[0].get("message", {}).get("content", "").strip()

            extracted_data.append({
                "custom_id": custom_id,
                "Claim_Detection": response
            })

# Creating and saving CSV
df = pd.DataFrame(extracted_data)
output_csv = "claim_detection_responses.csv"
df.to_csv(output_csv, index=False)

claim = (df["Claim_Detection"] == "1").sum()
no_claim = (df["Claim_Detection"] == "0").sum()

print(f"\nExtracted responses saved to: {output_csv}")
print(f"Contains claim    : {claim}")
print(f"No claim detected : {no_claim}")


Processing file-2LUjxme1c4bQxUwqQrseJU.jsonl
Processing file-LB5qVVZ8eacaUiAJyAwgd8.jsonl

Extracted responses saved to: claim_detection_responses.csv
Contains claim    : 26537
No claim detected : 44904
